# Introdução

Nesse notebook iremos fazer os downloads de todos os dados necessários para a nossa análise.

# Dados Oficiais do Citi Bike NY

In [1]:
import requests
import zipfile
from io import BytesIO
from pathlib import Path
import os
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
project_file = os.getcwd()

https://s3.amazonaws.com/tripdata/201601-citibike-tripdata.zip

In [ ]:
# Fazendo o download dos arquivos raw
year = '2022'
for month in range(1, 13):
    url = f"https://s3.amazonaws.com/tripdata/{year}{month:02d}-citibike-tripdata.csv.zip"
    print("Fazendo download de", url)
    req = requests.get(url)
    try:
        zf= zipfile.ZipFile(BytesIO(req.content))
        zf.extractall(f"{project_file}/Dados/CitiBikeRaw/")
    except Exception as e:
        print(f"Mês {month} não encontrado. Erro: {e}")

In [150]:
# Coletando o caminho de cada arquivo
citibikeraw_path = f"{project_file}\Dados\CitiBikeRaw"
file_paths = [f for f in listdir(citibikeraw_path) if isfile(join(citibikeraw_path, f))]

In [ ]:
# Salvando os dados em uma lista de dataframe
dfs = []
for datapath in file_paths:
    dfs.append(pd.read_csv(f"{citibikeraw_path}/{datapath}"))

In [152]:
# Compilandos toos meses em um unico arquivo
df_complete = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
# Salvando o dataframe do respectivo ano
df_complete.to_parquet(f"{project_file}/Dados/citibike{year}.parquet")

In [135]:
# Coletando amostra do original
df_complete.sample(1000000, random_state=42).to_parquet(f"{project_file}/Dados/Amostras/citibike{year}_sample.parquet")

In [159]:
for datapath in file_paths:
    os.remove(f"{citibikeraw_path}/{datapath}")

In [137]:
os.remove(f"{project_file}/Dados/citibike{year}.parquet")

## Compilando dos os Anos

In [138]:
# Salvando os dados em uma lista para serem compilados depois
amostras_path = f"{project_file}\Dados\Amostras"

file_paths = [f for f in listdir(amostras_path) if isfile(join(amostras_path, f))]

dfs = []
for datapath in file_paths:
    dfs.append(pd.read_parquet(f"{amostras_path}/{datapath}"))

In [139]:
# Compilando
df_complete = pd.concat(dfs, axis=0, ignore_index=True)
# Salvando
df_complete.to_parquet(f"{project_file}/Dados/citibike_limpo.parquet")

## Sujando o dataset

Negativando 30% dos anos de nascimento

In [140]:
linhas_com_nan_birth_year = df_complete[df_complete['Birth Year'].isna()]

# Obter amostras aleatórias da coluna 'Stop'
amostras_aleatorias = linhas_com_nan_birth_year.sample(frac=0.2, replace=True) 

# Multiplicar por -1 as amostras selecionadas da coluna 'Stop'
df_complete.loc[amostras_aleatorias.index, 'birth year'] = amostras_aleatorias['birth year'] * -1

Duplicando dados

In [141]:
linhas_dup = df_complete.sample(frac=0.25, replace=True)  # Porcentagem de linhas duplicadas desejadas (20% neste exemplo)
df_complete = pd.concat([df_complete, linhas_dup])

In [142]:
df_complete.duplicated().sum()

1000000

In [143]:
df_complete.to_parquet(f"{project_file}/Dados/citibike_sujo.parquet")